In [2]:
import numpy as np
import pandas as pd


In [3]:
rating = pd.read_csv("animelist.csv")

In [4]:
rating

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0
...,...,...,...,...,...
109224742,353404,507,7,2,2
109224743,353404,392,9,2,112
109224744,353404,882,6,2,1
109224745,353404,883,8,2,1


In [5]:
rating.describe()

,user_id,anime_id,rating,watching_status,watched_episodes
count,1.092247e+08,1.092247e+08,1.092247e+08,1.092247e+08,1.092247e+08
mean,1.768098e+05,1.649590e+04,4.245717e+00,3.087289e+00,1.210818e+01
std,1.018487e+05,1.379737e+04,3.912888e+00,1.774407e+00,1.463155e+02
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.849100e+04,3.194000e+03,0.000000e+00,2.000000e+00,0.000000e+00
50%,1.771420e+05,1.244500e+04,5.000000e+00,2.000000e+00,3.000000e+00
75%,2.651870e+05,3.083100e+04,8.000000e+00,6.000000e+00,1.200000e+01
max,3.534040e+05,4.849200e+04,1.000000e+01,5.500000e+01,6.553500e+04


In [6]:
rating_df = rating[["user_id","anime_id","rating"]]

In [7]:
rating_df.head()

,user_id,anime_id,rating
0,0,67,9
1,0,6702,7
2,0,242,10
3,0,4898,0
4,0,21,10


#### User must rate atleast 400 animes

In [8]:
n_rating = rating_df['user_id'].value_counts()
rating_df = rating_df[rating_df["user_id"].isin(n_rating[n_rating>=400].index)].copy()
len(rating_df)

71418114

### Scale the ratings between (0,1.0)

In [9]:
min_rating = min(rating_df['rating'])
max_rating = max(rating_df['rating'])
rating_df['rating'] = rating_df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values.astype(np.float64)



AvgRating = np.mean(rating_df['rating'])
print('Avgerage rating', AvgRating)

Avgerage rating 0.4047793589172634


### Remove duplicate rows

In [10]:
duplicate =  rating_df.duplicated()
if duplicate.sum()>0:
    rating_df = rating_df[~duplicate]

In [11]:
duplicate =  rating_df.duplicated()
duplicate.sum()

0

### Seeing top users and top anime. Ploting their crosstab

In [12]:
g = rating_df.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:10]
top_r = rating_df.join(top_users,rsuffix='_r',how='inner',on='user_id')

g = rating_df.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:10]
top_r = top_r.join(top_animes,rsuffix='_r',how='inner',on="anime_id")

pd.crosstab(top_r.user_id, top_r.anime_id,top_r.rating, aggfunc = np.sum)

anime_id,1535,1575,4224,5081,5114,6547,9253,11757,16498,19815
user_id,,,,,,,,,,
20807,1.0,1.0,1.0,0.9,1.0,1.0,1.0,1.0,1.0,1.0
85472,0.0,0.0,0.9,0.0,0.0,1.0,0.8,0.9,0.8,0.7
122341,0.3,0.2,0.3,0.3,0.8,0.5,0.7,0.1,0.1,0.4
131988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140590,0.6,0.5,0.8,0.8,0.9,0.5,0.9,0.4,0.5,0.7
147331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
281232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
283786,1.0,0.9,0.5,0.9,0.9,0.7,0.9,0.9,0.9,0.8
297931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0


## Data Preprocessing

In [13]:
user_ids = rating_df["user_id"].unique().tolist()

#### Encoding the user 

In [14]:
user2user = {x: i for i, x in enumerate(user_ids)}
encoded2user = {i: x for i, x in enumerate(user_ids)}

In [15]:
n_user = len(user2user)
print("Total users",n_user)

Total users 91641


In [16]:
anime_ids = rating_df["anime_id"].unique().tolist()
anime2anime =  {x: i for i, x in enumerate(anime_ids)}
encoded2anime = {i: x for i, x in enumerate(anime_ids)}

In [17]:
n_anime = len(anime2anime)
print('Total animes',n_anime)

Total animes 17560


In [18]:
rating_df["user"] = rating_df["user_id"].map(user2user)
rating_df["anime"] = rating_df["anime_id"].map(anime2anime)

#### Shuffle data

In [19]:
rating_df = rating_df.sample(frac=1, random_state=73)

X = rating_df[['user', 'anime']].values
y = rating_df["rating"]

#### Split data

In [20]:
test_set_size = 10000 #10k for test set
train_indices = rating_df.shape[0] - test_set_size 

X_train, X_test, y_train, y_test = (
    X[:train_indices],
    X[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [21]:
X_train

array([[23948, 11200],
       [72463,   757],
       [62985,  1808],
       ...,
       [34207,  3456],
       [58411,  3235],
       [58969,  2428]])

In [22]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_train_array

[array([23948, 72463, 62985, ..., 34207, 58411, 58969]),
 array([11200,   757,  1808, ...,  3456,  3235,  2428])]

In [23]:
X_test_array = [X_test[:, 0], X_test[:, 1]]
X_test_array

[array([12055, 12209, 63132, ..., 65683, 77931, 40652]),
 array([1513, 1437, 6488, ..., 1094, 1757, 3920])]

In [24]:
import tensorflow as tf

In [25]:
tf.__version__

'2.6.2'

In [26]:
import keras

In [27]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam 

In [28]:
# Embedding layers
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten

def RecommenderNet():
    embedding_size = 128
    
    user = Input(name = 'user', shape = [1])
    user_embedding = Embedding(name = 'user_embedding',
                       input_dim = n_user, 
                       output_dim = embedding_size)(user)
    
    anime = Input(name = 'anime', shape = [1])
    anime_embedding = Embedding(name = 'anime_embedding',
                       input_dim = n_anime, 
                       output_dim = embedding_size)(anime)
    
    #x = Concatenate()([user_embedding, anime_embedding])
    x = Dot(name = 'dot_product', normalize = True, axes = 2)([user_embedding, anime_embedding])
    x = Flatten()(x)
        
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("sigmoid")(x)
    
    model = Model(inputs=[user, anime], outputs=x)
    model.compile(loss='binary_crossentropy', metrics=["mae", "mse"], optimizer='Adam')
    
    return model

model = RecommenderNet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
anime (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 128)       11730048    user[0][0]                       
__________________________________________________________________________________________________
anime_embedding (Embedding)     (None, 1, 128)       2247680     anime[0][0]                      
______________________________________________________________________________________________

In [29]:

from tensorflow.keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005
batch_size = 10000


rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr


lr_callback = LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=0)

checkpoint_filepath = './weights.h5'

model_checkpoints = ModelCheckpoint(filepath=checkpoint_filepath,
                                        save_weights_only=True,
                                        monitor='val_loss',
                                        mode='min',
                                        save_best_only=True)

early_stopping = EarlyStopping(patience = 3, monitor='val_loss', 
                               mode='min', restore_best_weights=True)

my_callbacks = [
    model_checkpoints,
    lr_callback,
    early_stopping,   
]

In [31]:
history = model.fit(
    x=X_train_array,
    y=y_train,
    batch_size=batch_size,
    epochs=1,
    verbose=1,
    validation_data=(X_test_array, y_test),
    callbacks=my_callbacks
)

model.load_weights(checkpoint_filepath)

7141/7141 [==============================] - 3535s 494ms/step - loss: 0.7498 - mae: 0.3748 - mse: 0.1812 - val_loss: 0.7399 - val_mae: 0.3759 - val_mse: 0.1790


In [132]:
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

anime_weights = extract_weights('anime_embedding', model)
user_weights = extract_weights('user_embedding', model)

In [133]:
df = pd.read_csv('anime.csv', low_memory=True)
df = df.replace("Unknown", np.nan)

In [134]:
df.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [135]:
def getAnimeName(anime_id):
    try:
        name = df[df.anime_id == anime_id].eng_version.values[0]
        if name is np.nan:
            name = df[df.anime_id == anime_id].Name.values[0]
    except:
        print('error')
    
    return name

df['anime_id'] = df['MAL_ID']
df["eng_version"] = df['English name']
df['eng_version'] = df.anime_id.apply(lambda x: getAnimeName(x))

df.sort_values(by=['Score'], 
               inplace=True,
               ascending=False, 
               kind='quicksort',
               na_position='last')

df = df[["anime_id", "eng_version", "Score", "Genres", "Episodes", 
         "Type", "Premiered", "Members"]]

In [136]:
def getAnimeFrame(anime):
    if isinstance(anime, int):
        return df[df.anime_id == anime]
    if isinstance(anime, str):
        return df[df.eng_version == anime]

In [137]:
cols = ["MAL_ID", "Name", "Genres", "sypnopsis"]
sypnopsis_df = pd.read_csv('anime_with_synopsis.csv', usecols=cols)

def getSypnopsis(anime):
    if isinstance(anime, int):
        return sypnopsis_df[sypnopsis_df.MAL_ID == anime].sypnopsis.values[0]
    if isinstance(anime, str):
        return sypnopsis_df[sypnopsis_df.Name == anime].sypnopsis.values[0]

In [138]:
sypnopsis_df[sypnopsis_df["Name"]=="Tokyo Ghoul"]

,MAL_ID,Name,Genres,sypnopsis
7674,22319,Tokyo Ghoul,"Action, Mystery, Horror, Psychological, Supernatural, Drama, Seinen","Tokyo has become a cruel and merciless city—a place where vicious creatures called ""ghouls"" exist alongside humans. The citizens of this once great metropolis live in constant fear of these bloodthirsty savages and their thirst for human flesh. However, the greatest threat these ghouls pose is their dangerous ability to masquerade as humans and blend in with society. Based on the best-selling supernatural horror manga by Sui Ishida, Tokyo Ghoul follows Ken Kaneki, a shy, bookish college student, who is instantly drawn to Rize Kamishiro, an avid reader like himself. However, Rize is not exactly who she seems, and this unfortunate meeting pushes Kaneki into the dark depths of the ghouls' inhuman world. In a twist of fate, Kaneki is saved by the enigmatic waitress Touka Kirishima, and thus begins his new, secret life as a half-ghoul/half-human who must find a way to integrate into both societies."


In [158]:
pd.set_option("max_colwidth", None)

def find_similar_animes(name, n=10, return_dist=False, neg=False):
    try:
        
        index = getAnimeFrame(name).anime_id.values[0]
        #print(index)
        encoded_index = anime2anime.get(index)
        #print(encoded_index)
        weights = anime_weights
        
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        
        n = n + 1            
        
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]

        print('animes closest to {}'.format(name))
        if return_dist:
            return dists, closest
        
        rindex = df

        SimilarityArr = []
        
        for close in closest:
            decoded_id = encoded2anime.get(close)
            #print("here3")
            sypnopsis = getSypnopsis(decoded_id)
            anime_frame = getAnimeFrame(decoded_id)
            
            anime_name = anime_frame.eng_version.values[0]
            
            genre = anime_frame.Genres.values[0]
            similarity = dists[close]
            
            SimilarityArr.append({"anime_id": decoded_id, "name": anime_name,
                                  "similarity": similarity,"genre": genre,
                                  'sypnopsis': sypnopsis})

        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", ascending=False)
        return Frame[Frame.anime_id != index].drop(['anime_id'], axis=1)
    except Exception as e:
        print(e)
        print('{}!, Not Found in Anime list'.format(name))

In [159]:
find_similar_animes('Your Name.', n=5, neg=False)

animes closest to Your Name.


,name,similarity,genre,sypnopsis
4,Mazinger Z tai Ankoku Daishougun,0.337037,"Action, Mecha, Shounen","Kouji and his friends have defeated Dr. Hell and are now enjoying a break, but suddenly a strange prophet appears and warns everyone of an oncoming danger, mechanical beasts never seen before start appearing all around the world wrecking havoc. Its up to Kouji and his Mazinger Z to stand up to this threat but it seems he is vastly outnumber and outmatched. This film served as an alternative link between the Mazinger Z TV series and the Great Mazinger TV series. It basically introduces Great Mazinger to the audience, as well as his enemies from the Mikene Empire."
3,God Mars,0.305313,"Action, Mecha, Sci-Fi, Space","In the year 1999, humanity begins to advance beyond the solar system. The planet Gishin, led by the Emperor Zule, which aims to conquer the galaxy, runs into conflict with Earth. He targets Earth for elimination and to do this, he sends a baby called Mars to live among humanity. Accompanying the baby is a giant robot named Gaia, which utilizes a new power source strong enough to destroy an entire planet. As planned, Mars is expected to grow up, where he will activate the bomb within Gaia to fulfill the mission of destroying the Earth. However, when Mars arrives on Earth his is adopted into a Japanese family and given the name Takeru. Seventeen years later, Takeru would grow up with a love for humanity and refuses to detonate the bomb as ordered by Zule. However, if Takeru was to die, the bomb within Gaia would explode destroying the earth. Takeru possesses psychic powers ( ESP ) and decides to join the Earth defense forces and becomes a member of the Crasher Squad (an elite space defense force) where he and his friends take a last stand against the Gishin's attack. The relationship of Takeru with his brother Maag, which fate would have it, pitted the two against each other in the war. Unknown to the Gishin five other robots were created in secrecy along side Gaia by Takeru's father and sent with Gaia to protect Takeru. Whenever Earth is in danger, Takeru is able to summon the five other robots to combine with Gaia form the giant robot Godmars. The five other robots are Sphinx, Uranus, Titan, Shin and Ra."
2,Detective Conan Movie 17: Private Eye in the Distant Sea,0.301530,"Adventure, Mystery, Comedy, Police, Shounen","The warship Aegis Destroyer is conducting public exercises in Maizuru Bay where, coincidentally, a suspicious foreign ship was recently spotted. Conan Edogawa, Ran Mouri, Kogorou Mouri, Sonoko Suzuki, and the Detective Boys all receive a ticket to attend this event. However, while the ongoing military operations are underway, one of the crew members comes across a lieutenant's severed left arm. Conan later discovers that a foreign spy may have infiltrated the warship to obtain classified information by any means necessary. If the information were to leak, Japan's line of defense would be exposed, leaving the country unprotected from hostile attack. With the help of the police at sea while other friends and allies investigate on the mainland, Conan must now prevent this national crisis and identify the spy for the sake of Japan."
1,Chuubyou Gekihatsu Boy Special,0.299127,"Slice of Life, Comedy, School",Unaired episode 12 included with volume 4 of the BD/DVD release.
0,Hitotsu no Doa,0.290830,"Kids, Music",usic video for the song Hitotsu no Doa by Yui Komuro that was featured on NHK's Minna no Uta program. The video was animated by Keizou Kira.


In [160]:
find_similar_animes('Sword Art Online', n=10, neg=False)

animes closest to Sword Art Online


,name,similarity,genre,sypnopsis
9,Mahoromatic End Year Special,0.360323,"Ecchi, Romance, Comedy, Military, Sci-Fi",cap episode.
8,Tenchi Muyo! Ryo-Ohki,0.337389,"Action, Comedy, Harem, Sci-Fi, Shounen, Space","hile the galaxy remains at peace, Tenchi Masaki continues to live on Earth with Ryouko and Washuu Hakubi, Aeka and Sasami Jurai Masaki, Mihoshi Kuramitsu, and the cat-like creature Ryo-Ohki. Though it sounds great to be surrounded by so many women, they always end up making things difficult for Tenchi by constantly fighting for his attention, and their attempts at household chores always end in disaster. Although life has become more chaotic since they moved in, Tenchi begins to form a strong bond with his roommates and gains more insight into their respective struggles and history."
7,Bokurano,0.327979,"Sci-Fi, Psychological, Drama, Mecha, Seinen","15 children, 8 boys and 7 girls, are enjoying their summer camp together when they suddenly discover a grotto by the sea. When they enter the mysterious place they find a room full of computers, as well as a man named Kokopelli, who introduces himself as the owner. He claims to be working on a game which involves a giant robot that has been designed to protect the Earth from 15 different alien invasions. Kokopelli hasn't been able to test the game yet, so he persuades all but one of the children to sign a contract in what he claims will be a fun adventure. However, as soon as the contracts are signed things start to take a much darker turn. In Bokurano , the children must now pilot the giant robot Zearth one at a time in the hopes that they will have what it takes to defeat all of the upcoming enemies. But Kokopelli has left out one very important piece of information: the giant robot Zearth's energy source."
6,Sorcerer on the Rocks,0.306628,"Action, Comedy, Ecchi, Fantasy, Supernatural",uthless sorcerer Chivas Scotch is hired by a count named Cuttlefish to destroy an evil monster attacking a nearby village. But things go awry when Chivas fails to destroy the monster and he's forced to transfer his spirit in the body of his beautiful servant Gin Fizz. (Source: ANN)
5,Hengen Taima Yakou Karura Mau! Sendai Kokeshi Enka,0.299502,"Horror, Shoujo","Shoko and Maiko Ougi are apparently two ordinary schoolgirls in pursuit of graduating and having fun. Shii-chan is the more serious while Mai-chan is more fun-loving. In reality, the two sisters are powerful exorcists from the Karura temple. Each wields half the power... Shii-chan can ""see"" the spirits, and Mai-chan can banish them. This is a spooky series with heavy emphasis on traditionally Japanese occult themes. (Source: AnimeNfo)"
4,"Yume kara, Samenai",0.291600,"Romance, School",Student Takao is inexorably drawn toward Sao despite class rumors that she has appeared in a porno movie. (Source: AniDB)
3,Sore Ike! Anpanman: Gomira no Hoshi,0.291507,"Comedy, Kids, Fantasy",No synopsis information has been added to this title. Help improve our database by adding a synopsis here .
2,Run Melos,0.291039,"Historical, Drama","los, a Greek country man gets arrested accused of conspiracy against the king. The king gives him three days to travel to his sister's wedding while Selinentius the sculptor and friend of Melos stays as a hostage. Will he get back in time before the king executes Selinentius instead of him?"
1,ClassicaLoid,0.290966,"Comedy, Music","Ever since her father used up the last of the family fortune and skipped town, Kanae Otowa has lived in her family's ancestral mansion, Otowakan. With no way to pay off her father's debts, Otowakan is being foreclosed and Kanae must oust the remaining residents—her friend Sousuke Kagura, and two strange people who claim to be her father's acquaintances, Wolfgang Amadeus Mozart and Ludwig van Beethoven. After somehow being resurrected, Mozart and Beethoven have started going by ""Motes"" and ""Beethes"" respectively, and begun to indulge in a variety of idiotic, non-musical pursuits. While 

In [161]:
find_similar_animes('Black Clover', n=10, neg=False)

animes closest to Black Clover


,name,similarity,genre,sypnopsis
9,"Lupin the Third, The Woman Called Fujiko Mine",0.331964,"Action, Adventure, Comedy, Ecchi, Samurai, Seinen","any people are falling prey to a suspicious new religion. Lupin III infiltrates this group, hoping to steal the treasure their leader keeps hidden. There he lays eyes on the beautiful, bewitching woman who has the leader enthralled. This is the story of how fashionable female thief Fujiko Mine first met Lupin III, the greatest thief of his generation. (Source: ANN)"
8,Genius Party Beyond,0.326923,"Sci-Fi, Music, Dementia, Fantasy","Containing shorts that couldn't be included in the original, Genius Party Beyond weaves stories that are both deep and insightful: the idea all life is relative in size, the consequences of an oppressive government, and how to deal with your darker desires, among others. From the directors and artists of works such as Samurai Champloo , Neon Genesis Evangelion , and Akira , come a multitude of thought-provoking tales, to create a collection equalling the original in storytelling genius."
7,Yume wo Misete,0.323773,Music,usic video by Kousuke Sugimoto for the song Yume wo Misete by Yuuyake Shiwasu.
6,Season's Greetings from dwarf,0.303679,Kids,"Shorts celebrating the new 2017 and 2019 year and holidays, released by the animation studio dwarf."
5,"Clannad:Another World, Tomoyo Chapter",0.300685,"Drama, Romance, School, Slice of Life","Clannad: Mou Hitotsu no Sekai, Tomoyo-hen is set in an alternate reality where Tomoya Okazaki dates his junior, Tomoyo Sakagami. Tomoyo has been elected to be the school's next Student Council President. This is great news as she can now work toward her goal of preventing the school's cherry blossom trees from being axed. Although Tomoya is ecstatic for her, given his reputation as a delinquent in school, his relationship with Tomoyo is making them the subject of gossip around the campus, which can potentially compromise her standing as Student Council President. The school community's disapproval of their relationship becomes more apparent when the Student Council's Vice-President and even the school's administration warn Tomoya to distance himself from Tomoyo. With the bad atmosphere widening the rift between Tomoya and Tomoyo, will Tomoya succumb to societal pressure and do as they say, or will their love for each other rise above it all?"
4,Kekkero Ke,0.300299,Comedy,The anime follows the frogs Kekkero and Ke who try to find their 998 other siblings. (Source: ANN)
3,Black Jack:Capital Transfer To Heian,0.295694,Shounen,"These original short animated films are shown in the ""300 Inch Theater"" at the Tezuka Osamu World in the Kyoto Station Bldg, as well as a two-picture show including animated films from among Tezuka Osamu's major Manga, and one focuses on historic episodes of Kyoto. In this theatre, the Phoenix plays the role of storyteller, introducing two films. The Phoenix chooses a story entitled ""Old Woman"" (""Oba-chan"") from numerous episodes of ""Black Jack"" to discuss the theme of how much love can grow in this short lifetime. In the story, there is a seemingly greedy old woman who is always asking her daughter-in-law for money, and her son and his wife become fed up with the old woman's behaviour. Naturally, there are constant quarrels in the family, and their quiet ordinary life seems to have been lost. After accidentally coming to know the family, Black Jack learns that the old woman had her young son, who was dying at the time, treated and saved by a noted doctor who claimed a great amount of money. She has, in fact, been paying for the treatment over some tens of years. Because of her great love for her son, the old woman devoted her entire life to paying for the treatment. Not aware of his mother's act, the son feels helplessly hammered and squeezed by his mother and his wife. But when he finally comes to know the truth, he is deeply impressed with the love of his mother, who valued her son's life more than 